In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import pandas as pd
import yfinance as yf

In [2]:
df_2024 = pd.read_csv('coealg4uljn8tdpb.csv')
#ticker_series = df_2024[['permno', 'TICKER']].drop_duplicates(subset='permno')

df_2024_smol = df_2024[['public_date',
    'de_ratio', 'ptb', 'debt_assets','inv_turn','at_turn','rect_turn','cash_ratio',
    'pe_inc', 'gpm', 'roe', 'roa', # income statement
    'cfm', 'cash_debt', # cash flow
    'TICKER', 'permno'
]].dropna()

In [3]:
df_siccd_start = pd.read_csv('PERMNO_2_SICCD.csv')

In [4]:
# find common permno
df_2024_smol['permno'] = df_2024_smol['permno'].astype(int)
df_siccd_start['PERMNO'] = df_siccd_start['PERMNO'].astype(int)

# Step 1: Find the common PERMNOs between the two datasets
common_permnos = set(df_2024_smol['permno']).intersection(set(df_siccd_start['PERMNO']))

In [5]:
df_2024_smol_filtered = df_2024_smol[df_2024_smol['permno'].isin(common_permnos)]
df_siccd_start_filtered = df_siccd_start[df_siccd_start['PERMNO'].isin(common_permnos)]

print(len(df_2024_smol_filtered))
print(len(df_siccd_start_filtered))

del df_2024
del df_2024_smol
del df_siccd_start

296965
5034


In [6]:
df_siccd_start_filtered['starting_date'] = pd.to_datetime(df_siccd_start_filtered['starting_date'], errors='coerce')
df_siccd_start_filtered = df_siccd_start_filtered.sort_values(by=['PERMNO', 'starting_date'])

df_siccd_start_filtered['end_date'] = df_siccd_start_filtered.groupby('PERMNO')['starting_date'].shift(-1)
df_siccd_start_filtered['end_date'] = pd.to_datetime(df_siccd_start_filtered['end_date'], errors='coerce')

df_siccd_start_filtered


,Unnamed: 0,PERMNO,SICCD,starting_date,end_date
0,0,10001,4925,2010-01-04,NaT
2,2,10025,3081,2010-01-04,NaT
3,3,10026,2052,2010-01-04,NaT
4,4,10028,5094,2010-01-04,NaT
5,5,10032,3670,2010-01-04,NaT
...,...,...,...,...,...
19422,19422,93429,7374,2023-08-11,NaT
19427,19427,93432,7389,2010-06-08,NaT
19429,19429,93434,9999.0,2010-06-14,NaT
19430,19430,93435,6163,2010-06-14,NaT


In [7]:
df_merged = df_2024_smol_filtered.merge(df_siccd_start_filtered, left_on='permno', right_on='PERMNO', how='left')


df_merged = df_merged.rename(columns={'SICCD': 'tentativeSICCD'})


df_merged['public_date'] = pd.to_datetime(df_merged['public_date'], errors='coerce')
df_merged['starting_date'] = pd.to_datetime(df_merged['starting_date'], errors='coerce')

# Now apply the conditional logic after converting dates
df_merged['correct_siccd'] = df_merged.apply(
    lambda row: row['tentativeSICCD'] if (pd.isnull(row['end_date']) and row['public_date'] >= row['starting_date'])
    else (row['tentativeSICCD'] if row['starting_date'] <= row['public_date'] < row['end_date'] else None),
    axis=1
)

# Optionally filter rows where 'correct_siccd' is None
df_merged_filtered = df_merged.dropna(subset=['correct_siccd'])


print('len df_merged_filtered', len(df_merged_filtered))

# Display the filtered result
print(df_merged_filtered.head())

len df_merged_filtered 296965
  public_date  de_ratio    ptb  debt_assets  inv_turn  at_turn  rect_turn  \
0  2010-01-31     0.991  1.152        0.498     2.272    1.001      6.346   
2  2010-02-28     0.991  1.128        0.498     2.272    1.001      6.346   
4  2010-03-31     0.991  1.237        0.498     2.272    1.001      6.346   
6  2010-04-30     0.897  1.197        0.473     2.312    0.987      6.314   
8  2010-05-31     0.897  0.968        0.473     2.312    0.987      6.314   

   cash_ratio  pe_inc    gpm  ...    cfm  cash_debt  TICKER  permno  \
0       0.466  16.550  0.201  ...  0.073      0.089     AIR   54594   
2       0.466  16.200  0.201  ...  0.073      0.089     AIR   54594   
4       0.466  17.729  0.201  ...  0.073      0.089     AIR   54594   
6       0.545  19.821  0.195  ...  0.067      0.089     AIR   54594   
8       0.545  16.016  0.195  ...  0.067      0.089     AIR   54594   

  Unnamed: 0  PERMNO  tentativeSICCD  starting_date   end_date correct_siccd  
0

## construct daily merged filtered

In [12]:
df_merged_filtered = pd.read_csv('df_merged_filtered.csv')

/var/folders/_4/6qb0x2vs2lz5gz55b1_h00n80000gn/T/ipykernel_60546/1327075150.py:1: DtypeWarning: Columns (19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged_filtered = pd.read_csv('df_merged_filtered.csv')


In [9]:
earliest_dates = df_merged_filtered.groupby('TICKER')['public_date'].min().reset_index()

In [36]:
daily_returns_df = pd.read_csv('daily_returns.csv')
daily_returns_df['Date'] = pd.to_datetime(daily_returns_df['Date'])
daily_returns_df['month_key'] = daily_returns_df['Date'].dt.to_period('M')
daily_returns_df

,Unnamed: 0,TICKER,Date,Daily_Return,month_key
0,1,A,2010-02-02,0.012701,2010-02
1,2,A,2010-02-03,0.001355,2010-02
2,3,A,2010-02-04,-0.015910,2010-02
3,4,A,2010-02-05,0.003440,2010-02
4,5,A,2010-02-08,0.001029,2010-02
...,...,...,...,...,...
5646534,5648880,ZYXI,2024-09-23,-0.017241,2024-09
5646535,5648881,ZYXI,2024-09-24,0.016291,2024-09
5646536,5648882,ZYXI,2024-09-25,-0.014797,2024-09
5646537,5648883,ZYXI,2024-09-26,0.033792,2024-09


In [37]:
from datetime import timedelta

sp500 = yf.download('^GSPC', start=daily_returns_df['Date'].min()- timedelta(days=1), end=daily_returns_df['Date'].max(), progress=False)
vix = yf.download('^VIX', start=daily_returns_df['Date'].min(), end=daily_returns_df['Date'].max(), progress=False)

sp500 = sp500[['Adj Close']].rename(columns={'Adj Close': 'SP500_Close'}).reset_index()
vix = vix[['Adj Close']].rename(columns={'Adj Close': 'VIX_Close'}).reset_index()

sp500['SP500_Return'] = sp500['SP500_Close'].pct_change()
sp500 = sp500[['Date', 'SP500_Return']]

sp500['Date'] = pd.to_datetime(sp500['Date'])
vix['Date'] = pd.to_datetime(vix['Date'])


daily_returns_df = pd.merge(daily_returns_df, sp500, on='Date', how='left')
daily_returns_df = pd.merge(daily_returns_df, vix, on='Date', how='left')

daily_returns_df


,Unnamed: 0,TICKER,Date,Daily_Return,month_key,SP500_Return,VIX_Close
0,1,A,2010-02-02,0.012701,2010-02,0.012973,21.480000
1,2,A,2010-02-03,0.001355,2010-02,-0.005474,21.600000
2,3,A,2010-02-04,-0.015910,2010-02,-0.031141,26.080000
3,4,A,2010-02-05,0.003440,2010-02,0.002897,26.110001
4,5,A,2010-02-08,0.001029,2010-02,-0.008863,26.510000
...,...,...,...,...,...,...,...
5646534,5648880,ZYXI,2024-09-23,-0.017241,2024-09,0.002809,15.890000
5646535,5648881,ZYXI,2024-09-24,0.016291,2024-09,0.002511,15.390000
5646536,5648882,ZYXI,2024-09-25,-0.014797,2024-09,-0.001861,15.410000
5646537,5648883,ZYXI,2024-09-26,0.033792,2024-09,0.004039,15.370000


In [40]:
daily_returns_df = daily_returns_df.dropna(subset=['SP500_Return', 'VIX_Close'], how='all')

In [41]:
daily_returns_df

,Unnamed: 0,TICKER,Date,Daily_Return,month_key,SP500_Return,VIX_Close
0,1,A,2010-02-02,0.012701,2010-02,0.012973,21.480000
1,2,A,2010-02-03,0.001355,2010-02,-0.005474,21.600000
2,3,A,2010-02-04,-0.015910,2010-02,-0.031141,26.080000
3,4,A,2010-02-05,0.003440,2010-02,0.002897,26.110001
4,5,A,2010-02-08,0.001029,2010-02,-0.008863,26.510000
...,...,...,...,...,...,...,...
5646534,5648880,ZYXI,2024-09-23,-0.017241,2024-09,0.002809,15.890000
5646535,5648881,ZYXI,2024-09-24,0.016291,2024-09,0.002511,15.390000
5646536,5648882,ZYXI,2024-09-25,-0.014797,2024-09,-0.001861,15.410000
5646537,5648883,ZYXI,2024-09-26,0.033792,2024-09,0.004039,15.370000


In [42]:
df_merged_filtered['start_of_month'] = df_merged_filtered['public_date'].values.astype('datetime64[M]')
df_merged_filtered['end_of_month'] = (df_merged_filtered['start_of_month'] + pd.offsets.MonthEnd(0))


df_merged_filtered['month_key'] = df_merged_filtered['start_of_month'].dt.to_period('M')

df_merged_filtered

,Unnamed: 0.1,public_date,de_ratio,ptb,debt_assets,inv_turn,at_turn,rect_turn,cash_ratio,pe_inc,...,permno,Unnamed: 0,PERMNO,tentativeSICCD,starting_date,end_date,correct_siccd,start_of_month,end_of_month,month_key
0,0,2010-01-31,0.991,1.152,0.498,2.272,1.001,6.346,0.466,16.550,...,54594,11702,54594,3721.0,2010-01-04,2014-01-08,3721.0,2010-01-01,2010-01-31,2010-01
1,2,2010-02-28,0.991,1.128,0.498,2.272,1.001,6.346,0.466,16.200,...,54594,11702,54594,3721.0,2010-01-04,2014-01-08,3721.0,2010-02-01,2010-02-28,2010-02
2,4,2010-03-31,0.991,1.237,0.498,2.272,1.001,6.346,0.466,17.729,...,54594,11702,54594,3721.0,2010-01-04,2014-01-08,3721.0,2010-03-01,2010-03-31,2010-03
3,6,2010-04-30,0.897,1.197,0.473,2.312,0.987,6.314,0.545,19.821,...,54594,11702,54594,3721.0,2010-01-04,2014-01-08,3721.0,2010-04-01,2010-04-30,2010-04
4,8,2010-05-31,0.897,0.968,0.473,2.312,0.987,6.314,0.545,16.016,...,54594,11702,54594,3721.0,2010-01-04,2014-01-08,3721.0,2010-05-01,2010-05-31,2010-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296960,421848,2022-08-31,0.397,0.375,0.284,2.060,0.553,11.357,0.229,-0.688,...,20194,7306,20194,9999.0,2020-12-10,NaN,9999.0,2022-08-01,2022-08-31,2022-08
296961,421849,2022-09-30,0.397,0.214,0.284,2.060,0.553,11.357,0.229,-0.393,...,20194,7306,20194,9999.0,2020-12-10,NaN,9999.0,2022-09-01,2022-09-30,2022-09
296962,421850,2022-10-31,0.397,0.285,0.284,2.060,0.553,11.357,0.229,-0.522,...,20194,7306,20194,9999.0,2020-12-10,NaN,9999.0,2022-10-01,2022-10-31,2022-10
296963,421851,2022-11-30,0.478,0.296,0.324,1.985,0.521,10.877,0.290,-0.431,...,20194,7306,20194,9999.0,2020-12-10,NaN,9999.0,2022-11-01,2022-11-30,2022-11


In [43]:
df_combined = pd.merge(
    daily_returns_df, 
    df_merged_filtered, 
    on=['TICKER', 'month_key'], 
    suffixes=('_daily', '_monthly')
)

df_combined = df_combined[
    (df_combined['Date'] >= df_combined['start_of_month']) & 
    (df_combined['Date'] <= df_combined['end_of_month'])
]

## algorithm

In [44]:
from tqdm.notebook import tqdm

available_dates_per_ticker = {}
total_available_dates = []

# Group by ticker to process one ticker at a time
tickers_grouped = df_combined.groupby('TICKER')

for ticker, ticker_data in tqdm(tickers_grouped):
    ticker_data = ticker_data.sort_values(by='Date')
    dts = ticker_data.iloc[5:-2].Date
    available_dates_per_ticker[ticker] = dts
    total_available_dates.extend(list(dts))
    

  0%|          | 0/2144 [00:00<?, ?it/s]

In [45]:
total_available_dates = sorted(list(set(total_available_dates)))

# Initialize an empty dictionary for the inverted index
date_to_tickers = {}

for ticker, dates in available_dates_per_ticker.items():
    date_to_index = {date: idx for date, idx in zip(dates, dates.index)}

    # Build the inverted index
    for date, idx in date_to_index.items():
        if date not in date_to_tickers:
            date_to_tickers[date] = []
        date_to_tickers[date].append((ticker, idx))

# Now, date_to_tickers is an inverted index where each date maps to a list of tickers

# Example: Query for tickers available on a specific central_date
central_date = pd.Timestamp('2020-09-25')
tickers_on_central_date = date_to_tickers.get(central_date, [])
print(f'Tickers available on {central_date}: {tickers_on_central_date}')


Tickers available on 2020-09-25 00:00:00: [('A', 2681), ('AA', 5810), ('AAOI', 9251), ('AAP', 15251), ('AAPL', 18503), ('ABG', 26770), ('ABT', 30527), ('ACA', 31707), ('ACAD', 33178), ('ACCO', 35655), ('ACHC', 39327), ('ACLS', 43146), ('ACMR', 44323), ('ACRS', 46007), ('ACU', 48987), ('ADI', 53205), ('ADM', 56079), ('ADMA', 57867), ('ADPT', 58688), ('ADT', 59865), ('ADTN', 63117), ('AE', 67045), ('AEE', 70297), ('AEHR', 73296), ('AEIS', 76948), ('AEO', 80200), ('AEP', 83452), ('AES', 86704), ('AGCO', 90754), ('AGIO', 92666), ('AGR', 94158), ('AGS', 95587), ('AGYS', 101587), ('AIHS', 102953), ('AIN', 107019), ('AIR', 110271), ('AIRG', 111678), ('AIRI', 114027), ('AIRT', 117216), ('AIT', 120468), ('AKBA', 121709), ('AKTS', 122988), ('ALB', 126388), ('ALCO', 129640), ('ALE', 132892), ('ALG', 138001), ('ALGN', 141717), ('ALGT', 144969), ('ALK', 148221), ('ALNY', 149248), ('ALOT', 154618), ('ALRM', 156425), ('ALSN', 159089), ('ALTR', 161741), ('ALV', 164762), ('AMAT', 167950), ('AMC', 17017

In [46]:
import numpy as np
def siccd_vectorize(string):
    try:
        string = str(int(float(string)))
        if len(string)==4:
            return np.array([int(string[0]),
                             int(string[:2]),
                             int(string[:3]),
                             int(string[:4])])
        elif len(string)==3:
            return np.array([int(string[0]),
                             int(string[:2]),
                             int(string[:3]),
                             10*int(string[:3])])
    except:
        return np.array([9,99,999,9999])

In [48]:
cd = 2225

In [49]:
def six_to_two(df_slice,
              ts_cols=['de_ratio', 'ptb', 'debt_assets', 'inv_turn', 'at_turn', 'rect_turn', 
                  'cash_ratio', 'pe_inc', 'gpm', 'roe', 'roa', 'cfm', 'cash_debt','SP500_Return', 'VIX_Close','Daily_Return'],
              ):
    if len(df_slice) == 6:
        return df_slice[ts_cols].iloc[:5].values, siccd_vectorize(df_slice.correct_siccd.mode().iloc[0]), df_slice.Daily_Return.iloc[5]
    else:
        print('error')
        display(df_slice)

six_to_two(df_combined.iloc[cd-5:cd+1])

(array([[ 8.48000000e-01,  5.05300000e+00,  4.59000000e-01,
          3.33000000e+00,  5.62000000e-01,  6.49400000e+00,
          2.14000000e+00,  7.95050000e+01,  5.85000000e-01,
          6.40000000e-02,  1.35000000e-01,  1.05000000e-01,
          2.48000000e-01,  1.55323684e-02,  1.88999996e+01,
          1.52469146e-02],
        [ 8.48000000e-01,  5.05300000e+00,  4.59000000e-01,
          3.33000000e+00,  5.62000000e-01,  6.49400000e+00,
          2.14000000e+00,  7.95050000e+01,  5.85000000e-01,
          6.40000000e-02,  1.35000000e-01,  1.05000000e-01,
          2.48000000e-01,  3.26169214e-03,  1.90200005e+01,
          1.58835616e-03],
        [ 8.48000000e-01,  5.05300000e+00,  4.59000000e-01,
          3.33000000e+00,  5.62000000e-01,  6.49400000e+00,
          2.14000000e+00,  7.95050000e+01,  5.85000000e-01,
          6.40000000e-02,  1.35000000e-01,  1.05000000e-01,
          2.48000000e-01,  2.29739796e-02,  1.84899998e+01,
          3.05653278e-02],
        [ 8.4800000

In [50]:
Xs = []
X_stat_s = []
ys = []

from tqdm.notebook import tqdm

for central_date in tqdm(total_available_dates):
    list_ticker_row = date_to_tickers[central_date]
    
    X_day = []
    X_stat_day = []
    y_day = []
    
    for ticker,row in list_ticker_row:
        try:
            X,X_stat,y = six_to_two(df_combined.iloc[row-5:row+1])
            
            X_day.append(X)
            X_stat_day.append(X_stat)
            y_day.append(y)
        except:
            print(ticker, row)
            pass
        
    print(central_date, len(X_day), len(X_stat_day), len(y_day))
        
    Xs.append(np.array(X_day))
    X_stat_s.append(np.array(X_stat_day))
    ys.append(np.array(y_day))

  0%|          | 0/3245 [00:00<?, ?it/s]

2010-02-09 00:00:00 982 982 982
2010-02-10 00:00:00 983 983 983
2010-02-11 00:00:00 984 984 984
2010-02-12 00:00:00 984 984 984
2010-02-16 00:00:00 984 984 984
2010-02-17 00:00:00 984 984 984
2010-02-18 00:00:00 985 985 985
2010-02-19 00:00:00 985 985 985
2010-02-22 00:00:00 985 985 985
2010-02-23 00:00:00 985 985 985
2010-02-24 00:00:00 984 984 984
2010-02-25 00:00:00 983 983 983
2010-02-26 00:00:00 982 982 982
2010-03-01 00:00:00 979 979 979
2010-03-02 00:00:00 980 980 980
2010-03-03 00:00:00 981 981 981
2010-03-04 00:00:00 981 981 981
2010-03-05 00:00:00 980 980 980
2010-03-08 00:00:00 981 981 981
2010-03-09 00:00:00 1002 1002 1002
2010-03-10 00:00:00 1002 1002 1002
2010-03-11 00:00:00 1002 1002 1002
2010-03-12 00:00:00 1002 1002 1002
2010-03-15 00:00:00 1001 1001 1001
2010-03-16 00:00:00 1002 1002 1002
2010-03-17 00:00:00 1002 1002 1002
2010-03-18 00:00:00 1002 1002 1002
2010-03-19 00:00:00 1001 1001 1001
2010-03-22 00:00:00 1000 1000 1000
2010-03-23 00:00:00 1002 1002 1002
2010-03

2011-01-19 00:00:00 1039 1039 1039
2011-01-20 00:00:00 1041 1041 1041
2011-01-21 00:00:00 1040 1040 1040
2011-01-24 00:00:00 1040 1040 1040
2011-01-25 00:00:00 1040 1040 1040
2011-01-26 00:00:00 1041 1041 1041
2011-01-27 00:00:00 1040 1040 1040
2011-01-28 00:00:00 1039 1039 1039
2011-01-31 00:00:00 1038 1038 1038
2011-02-01 00:00:00 1017 1017 1017
2011-02-02 00:00:00 1017 1017 1017
2011-02-03 00:00:00 1017 1017 1017
2011-02-04 00:00:00 1018 1018 1018
2011-02-07 00:00:00 1017 1017 1017
2011-02-08 00:00:00 1019 1019 1019
2011-02-09 00:00:00 1018 1018 1018
2011-02-10 00:00:00 1020 1020 1020
2011-02-11 00:00:00 1019 1019 1019
2011-02-14 00:00:00 1019 1019 1019
2011-02-15 00:00:00 1018 1018 1018
2011-02-16 00:00:00 1018 1018 1018
2011-02-17 00:00:00 1020 1020 1020
2011-02-18 00:00:00 1018 1018 1018
2011-02-22 00:00:00 1019 1019 1019
2011-02-23 00:00:00 1019 1019 1019
2011-02-24 00:00:00 1019 1019 1019
2011-02-25 00:00:00 1016 1016 1016
2011-02-28 00:00:00 1016 1016 1016
2011-03-01 00:00:00 

2011-12-23 00:00:00 1060 1060 1060
2011-12-27 00:00:00 1059 1059 1059
2011-12-28 00:00:00 1060 1060 1060
2011-12-29 00:00:00 1060 1060 1060
2011-12-30 00:00:00 1059 1059 1059
2012-01-03 00:00:00 1059 1059 1059
2012-01-04 00:00:00 1059 1059 1059
2012-01-05 00:00:00 1059 1059 1059
2012-01-06 00:00:00 1059 1059 1059
2012-01-09 00:00:00 1059 1059 1059
2012-01-10 00:00:00 1059 1059 1059
2012-01-11 00:00:00 1060 1060 1060
2012-01-12 00:00:00 1060 1060 1060
2012-01-13 00:00:00 1060 1060 1060
2012-01-17 00:00:00 1060 1060 1060
2012-01-18 00:00:00 1060 1060 1060
2012-01-19 00:00:00 1060 1060 1060
2012-01-20 00:00:00 1060 1060 1060
2012-01-23 00:00:00 1060 1060 1060
2012-01-24 00:00:00 1060 1060 1060
2012-01-25 00:00:00 1060 1060 1060
2012-01-26 00:00:00 1060 1060 1060
2012-01-27 00:00:00 1060 1060 1060
2012-01-30 00:00:00 1056 1056 1056
2012-01-31 00:00:00 1056 1056 1056
2012-02-01 00:00:00 1047 1047 1047
2012-02-02 00:00:00 1047 1047 1047
2012-02-03 00:00:00 1047 1047 1047
2012-02-06 00:00:00 

2012-12-04 00:00:00 1076 1076 1076
2012-12-05 00:00:00 1076 1076 1076
2012-12-06 00:00:00 1076 1076 1076
2012-12-07 00:00:00 1075 1075 1075
2012-12-10 00:00:00 1088 1088 1088
2012-12-11 00:00:00 1088 1088 1088
2012-12-12 00:00:00 1088 1088 1088
2012-12-13 00:00:00 1088 1088 1088
2012-12-14 00:00:00 1087 1087 1087
2012-12-17 00:00:00 1087 1087 1087
2012-12-18 00:00:00 1088 1088 1088
2012-12-19 00:00:00 1088 1088 1088
2012-12-20 00:00:00 1088 1088 1088
2012-12-21 00:00:00 1087 1087 1087
2012-12-24 00:00:00 1087 1087 1087
2012-12-26 00:00:00 1086 1086 1086
2012-12-27 00:00:00 1088 1088 1088
2012-12-28 00:00:00 1086 1086 1086
2012-12-31 00:00:00 1087 1087 1087
2013-01-02 00:00:00 1087 1087 1087
2013-01-03 00:00:00 1087 1087 1087
2013-01-04 00:00:00 1085 1085 1085
2013-01-07 00:00:00 1087 1087 1087
2013-01-08 00:00:00 1086 1086 1086
2013-01-09 00:00:00 1088 1088 1088
2013-01-10 00:00:00 1087 1087 1087
2013-01-11 00:00:00 1087 1087 1087
2013-01-14 00:00:00 1088 1088 1088
2013-01-15 00:00:00 

...

2022-02-03 00:00:00 1490 1490 1490
2022-02-04 00:00:00 1490 1490 1490
2022-02-07 00:00:00 1490 1490 1490
2022-02-08 00:00:00 1493 1493 1493
2022-02-09 00:00:00 1493 1493 1493
2022-02-10 00:00:00 1493 1493 1493
2022-02-11 00:00:00 1493 1493 1493
2022-02-14 00:00:00 1493 1493 1493
2022-02-15 00:00:00 1493 1493 1493
2022-02-16 00:00:00 1493 1493 1493
2022-02-17 00:00:00 1493 1493 1493
2022-02-18 00:00:00 1493 1493 1493
2022-02-22 00:00:00 1493 1493 1493
2022-02-23 00:00:00 1493 1493 1493
2022-02-24 00:00:00 1493 1493 1493
2022-02-25 00:00:00 1489 1489 1489
2022-02-28 00:00:00 1489 1489 1489
2022-03-01 00:00:00 1485 1485 1485
2022-03-02 00:00:00 1485 1485 1485
2022-03-03 00:00:00 1485 1485 1485
2022-03-04 00:00:00 1485 1485 1485
2022-03-07 00:00:00 1485 1485 1485
2022-03-08 00:00:00 1534 1534 1534
2022-03-09 00:00:00 1534 1534 1534
2022-03-10 00:00:00 1534 1534 1534
2022-03-11 00:00:00 1534 1534 1534
2022-03-14 00:00:00 1534 1534 1534
2022-03-15 00:00:00 1534 1534 1534
2022-03-16 00:00:00 

/var/folders/_4/6qb0x2vs2lz5gz55b1_h00n80000gn/T/ipykernel_60546/43720542.py:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  return df_slice[ts_cols].iloc[:5].values, siccd_vectorize(df_slice.correct_siccd.mode().iloc[0]), df_slice.Daily_Return.iloc[5]


2022-05-04 00:00:00 1568 1568 1568
2022-05-05 00:00:00 1568 1568 1568
2022-05-06 00:00:00 1568 1568 1568
2022-05-09 00:00:00 1571 1571 1571
2022-05-10 00:00:00 1571 1571 1571
2022-05-11 00:00:00 1571 1571 1571
2022-05-12 00:00:00 1571 1571 1571
2022-05-13 00:00:00 1571 1571 1571
2022-05-16 00:00:00 1571 1571 1571
2022-05-17 00:00:00 1571 1571 1571
2022-05-18 00:00:00 1571 1571 1571
2022-05-19 00:00:00 1571 1571 1571
2022-05-20 00:00:00 1571 1571 1571
2022-05-23 00:00:00 1571 1571 1571
2022-05-24 00:00:00 1571 1571 1571
2022-05-25 00:00:00 1571 1571 1571
2022-05-26 00:00:00 1571 1571 1571
2022-05-27 00:00:00 1570 1570 1570
2022-05-31 00:00:00 1570 1570 1570
2022-06-01 00:00:00 1573 1573 1573
2022-06-02 00:00:00 1573 1573 1573
2022-06-03 00:00:00 1573 1573 1573
2022-06-06 00:00:00 1573 1573 1573
2022-06-07 00:00:00 1573 1573 1573
2022-06-08 00:00:00 1656 1656 1656
2022-06-09 00:00:00 1656 1656 1656
2022-06-10 00:00:00 1656 1656 1656
2022-06-13 00:00:00 1656 1656 1656
2022-06-14 00:00:00 

/var/folders/_4/6qb0x2vs2lz5gz55b1_h00n80000gn/T/ipykernel_60546/43720542.py:6: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'float'
  return df_slice[ts_cols].iloc[:5].values, siccd_vectorize(df_slice.correct_siccd.mode().iloc[0]), df_slice.Daily_Return.iloc[5]


2022-11-03 00:00:00 1676 1676 1676
2022-11-04 00:00:00 1676 1676 1676
2022-11-07 00:00:00 1676 1676 1676
2022-11-08 00:00:00 1677 1677 1677
2022-11-09 00:00:00 1677 1677 1677
2022-11-10 00:00:00 1677 1677 1677
2022-11-11 00:00:00 1677 1677 1677
2022-11-14 00:00:00 1677 1677 1677
2022-11-15 00:00:00 1677 1677 1677
2022-11-16 00:00:00 1677 1677 1677
2022-11-17 00:00:00 1677 1677 1677
2022-11-18 00:00:00 1677 1677 1677
2022-11-21 00:00:00 1677 1677 1677
2022-11-22 00:00:00 1677 1677 1677
2022-11-23 00:00:00 1677 1677 1677
2022-11-25 00:00:00 1677 1677 1677
2022-11-28 00:00:00 1677 1677 1677
2022-11-29 00:00:00 1674 1674 1674
2022-11-30 00:00:00 1674 1674 1674
2022-12-01 00:00:00 1675 1675 1675
2022-12-02 00:00:00 1675 1675 1675
2022-12-05 00:00:00 1675 1675 1675
2022-12-06 00:00:00 1675 1675 1675
2022-12-07 00:00:00 1675 1675 1675
2022-12-08 00:00:00 1706 1706 1706
2022-12-09 00:00:00 1706 1706 1706
2022-12-12 00:00:00 1706 1706 1706
2022-12-13 00:00:00 1706 1706 1706
2022-12-14 00:00:00 

In [51]:
import pickle

with open('Gopal_Xs.pkl', 'wb') as f:
    pickle.dump(Xs, f)
with open('Gopal_X_stat_s.pkl', 'wb') as f:
    pickle.dump(X_stat_s, f)
with open('Gopal_ys.pkl', 'wb') as f:
    pickle.dump(ys,f)